#  ColumnTransformer 완벽 정리 (scikit-learn)

##  ColumnTransformer란?

`ColumnTransformer`는 **열(Column)마다 다른 전처리기를 적용**할 수 있게 해주는 scikit-learn의 유틸리티입니다.

> 서로 다른 데이터 타입(수치형, 범주형, 텍스트 등)에 대해 **각기 다른 전처리**를 적용하고,
> 이를 하나의 파이프라인 안에서 효율적이고 안전하게 구성할 수 있습니다.

---

##  왜 필요한가?

| 열 이름 | 데이터 타입 | 처리 방식 예시            |
|--------|--------------|----------------------------|
| age     | 수치형        | 평균 보간 → 정규화         |
| income  | 수치형        | 정규화 (StandardScaler)    |
| gender  | 범주형        | OneHot 인코딩             |
| job     | 범주형        | OneHot or Ordinal 인코딩  |

➡ `ColumnTransformer`를 사용하면 열마다 전처리기를 따로 지정할 수 있습니다.

---

##  기본 구조

```python
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'income']),
        ('cat', OneHotEncoder(), ['gender', 'job'])
    ],
    remainder='drop'  # 지정되지 않은 열은 버림
)


ColumnTransformer + Pipeline 실습 예제 (Titanic)

In [1]:
#데이터 불러오기
from sklearn.datasets import fetch_openml
import pandas as pd

# Titanic 데이터셋 로드
titanic = fetch_openml('titanic', version=1, as_frame=True)
df = titanic.frame.copy()

# 대상 변수와 특성 나누기
X = df[['pclass', 'sex', 'age', 'fare']]
y = df['survived']

In [2]:
#전처리 구성
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 특성 분리
numeric_features = ['age', 'fare']
categorical_features = ['pclass', 'sex']

# 전처리기
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])


전체 파이프라인 구성 + 학습

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 파이프라인 구성
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# 모델 학습
clf.fit(X_train, y_train)

# 평가
print("테스트 정확도:", clf.score(X_test, y_test))

테스트 정확도: 0.8109756097560976
